# Debugging the CW308_STM32F3 Using ChipWhisperer

## Background

When testing software, it can be extremely helpful to view and modify the state of the software while it is running. For programs running on fully featured operating systems (Windows, Mac, Linux, etc.), this can be accomplished by using debugging programs, such as `gdb`. Microcontrollers and FPGAs, on the other hand, require additional hardware to debug. While it is better to use a device dedicated to debugging, beginning with ChipWhisperer 5.6.1, you can use your ChipWhisperer capture board to debug target devices. This is done over SWD/JTAG, which most devices support. To see if your target supports JTAG/SWD, you should check the reference manual/datasheet for its microcontroller.

This demo will run will show you how to set your hardware up for debugging, how to set the software up for debugging, and finally how to program and debug an STM32F3. In addition to this demo, you may want to skip through and have open our [debugging documentation](https://chipwhisperer.readthedocs.io/en/latest/debugging.html).

## Checking for ChipWhisperer Support

This demo requires ChipWhisperer 5.6.1. In addition, you must have firmware from this version of ChipWhisperer or newer running on your device. Run the following line to check for support:

In [1]:
import sys
if '../../software' not in sys.path:
    sys.path.append('../../software')
if '../courses/fault101/python-dilithium/' not in sys.path:
    sys.path.append('../courses/fault101/python-dilithium/')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/demos',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../software',
 '../courses/fault101/python-dilithium/']

In [2]:
scope.dis()

NameError: name 'scope' is not defined

In [7]:
import chipwhisperer as cw
if cw.__version__ < '5.6.1':
    print("MPSSE not supported.")
else:
    #help(scope)
    scope = cw.scope()
    print(type(scope))
    if scope.check_feature("MPSSE"):
        print("MPSSE supported!")
    else:
        print("MPSSE not supported. checkfea")
    scope.dis()

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


<class 'chipwhisperer.capture.scopes.OpenADC.OpenADC'>
MPSSE not supported. checkfea


In [38]:
import chipwhisperer as cw
if cw.__version__ < '5.6.1':
    print("MPSSE not supported.")
else:
    #help(scope)
    scope = cw.scope()
    print(type(scope))
    if scope.check_feature("MPSSE"):
        print("MPSSE supported!")
    else:
        print("MPSSE not supported. checkfea")
    scope.dis()

<class 'chipwhisperer.capture.scopes.OpenADC.OpenADC'>
MPSSE supported!


In [36]:
import time
from chipwhisperer.hardware.naeusb.bootloader_sam3u import Samba
from chipwhisperer.hardware.firmware.cwlite import getsome
class Poo:
    def __init__(self):
        self.logfunc = print
        
    def program(self, port='/dev/ttyACM0', hardware_type='cwlite'):
        name = 'SAM3U_CW1173.bin'
        fw_data = getsome(name).read()
        #print(fw_data)
        #return
        sam = Samba()
        print(port)
        sam.con(port)
        self.logfunc("Connected!\nErasing...")
        sam.erase()
        self.logfunc("Erased!\nProgramming file {}...".format(name))
        sam.write(fw_data)
        self.logfunc("Programmed!\nVerifying...")
        if sam.verify(fw_data):
            self.logfunc("Verify OK!")
            sam.flash.setBootFlash(1)

            i = 0

            while not sam.flash.getBootFlash():
                sam.flash.setBootFlash(1)
                if sam.flash.name == "ATSAM4S2":
                    break #IIRC there's a bug on the SAM4S that prevents this from being read TODO: check errata
                time.sleep(0.05)
                i += 1
                if i > 10:
                    sam.reset()
                    sam.ser.close()
                    self.logfunc("Upgrade succeded")
                    self.logfunc("Unable to set boot flash, may need to power cycle")
                    return True

            self.logfunc("Resetting...")
            sam.reset()
            self.logfunc("Upgrade successful")
            sam.ser.close()
            return True
        else:
            self.logfunc("Verify FAILED!")
            sam.ser.close()
            return False

In [37]:
o = Poo()
ret = o.program()
ret

/dev/ttyACM0
Connected!
Erasing...
Erased!
Programming file SAM3U_CW1173.bin...
Programmed!
Verifying...
Verify OK!
Resetting...
Upgrade successful


True

In [14]:
scope = cw.scope()
scope.upgrade_firmware()

OSError: Could not find ChipWhisperer. Is it connected?

## Software Prerequisites

In addition to ChipWhisperer, you will need the following software:

1. OpenOCD, available at https://openocd.org/pages/getting-openocd.html. It's recommended that you download an OpenOCD binary or built it yourself, rather than getting it through a package manager, as the package manager will usually install an old version of OpenOCD
1. GDB for your device of choice (`arm-none-eabi-gdb`, in our case). This is typically included with the compiler you use to build firmware for your target.

Ensure both openocd and gdb are available on your path:

* Windows: https://www.computerhope.com/issues/ch000549.htm (set in user variables)
* Mac/Linux: https://superuser.com/questions/488173/how-can-i-edit-the-path-on-linux

Once you've got those installed, quit jupyter and reopen it in a new terminal window so that your path updates. Make sure you're able to access both of these programs:

In [43]:
if '/home/alexis/bin' not in sys.path:
    sys.path.append('/home/alexis/bin')

In [45]:
%%bash
arm-none-eabi-gdb --version

bash: line 1: arm-none-eabi-gdb: command not found


CalledProcessError: Command 'b'arm-none-eabi-gdb --version\n'' returned non-zero exit status 127.

In [42]:
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/demos',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../software',
 '../courses/fault101/python-dilithium/']

In [ ]:
%%bash
openocd --version

## Hardware Setup

### ChipWhisperer-Lite, Pro, Nano, or Husky with CW308T_STM32F3

1. Connect your ChipWhisperer capture board to your PC
1. Connect your capture board and CW308 via the CW 20-pin connector
1. Use jumper wires to connect the following pins on the CW308:
    1. J5 Pin 12 (SCK)  -> J8 Pin 6 (J_TCK)
    1. J5 Pin 18 (PDID) -> J8 Pin 5 (J_TMS)
    1. J5 Pin 13 (MISO) -> J8 Pin 4 (J_TDO)
    1. J5 Pin 14 (MOSI) -> J8 Pin 3 (J_TDI)
    
<img src="img/MPSSE_con_1x.jpg" alt="MPSSE-DIO" width=600>
    
### ChipWhisperer-Husky with CW308T_STM32F3 using 20-pin DIO Header

1. Connect your ChipWhisperer capture board to your PC
1. Connect your capture board and CW308 via the CW 20-pin connector
1. Connect the 20-pin DIO header to J6 on the CW308. Ensure the keyed part of the connector is facing away from the board

<img src="img/MPSSE_con_3.jpg" alt="MPSSE-DIO" width=600>

### ChipWhisperer-Lite, Pro, Nano, or Husky with CW312T_STM32F3

Coming soon!

## Software Setup

### Lite, Pro, Nano, or Husky (non DIO) Setup

All you need to do to enable MPSSE is connect to your scope and run `scope.enable_MPSSE(1)` (we'll also record the CW's USB PID for later usage):

In [ ]:
import chipwhisperer as cw
scope = cw.scope()
PID = hex(scope._getNAEUSB().usbtx.device.getProductID())
scope.enable_MPSSE(1)
scope.dis()

###  Husky DIO Setup

Alternatively, if you want to use the ChipWhisperer-Husky's 20-pin DIO header, enable that before running `scope.enable_MPSSE(1)`:

In [ ]:
import chipwhisperer as cw
scope = cw.scope()
PID = hex(scope._getNAEUSB().usbtx.device.getProductID())
scope.enable_MPSSE(1, husky_userio='jtag') # or husky_userio='swd'

With that done, we can now debug our target. Note that `scope.enable_MPSSE(1)` will disconnect your target in Python, meaning you'll have to reconnect to interact via the API. On Windows, only one process at a time can access a USB device, meaning if you reconnect in Python, you won't be able to connect in OpenOCD and vice versa.

We'll start by building and uploading some firmware to the target:

In [ ]:
%%bash -s "$PID"

# build firmware
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=CW308_STM32F3 CRYPTO_TARGET=TINYAES128C SS_VER=SS_VER_1_1 -j

# upload firmware via JTAG
openocd -f ../../../../openocd/cw_openocd.cfg -c "transport select jtag" -c "ftdi vid_pid 0x2b3e $1" -f "target/stm32f3x.cfg" -c "init" -c "targets" -c "halt" -c "flash write_image erase simpleserial-aes-CW308_STM32F3.elf" -c "flash verify_image simpleserial-aes-CW308_STM32F3.elf" -c "reset run" -c "shutdown"

If you don't see any errors, then congratulations: You've now used JTAG to upload code onto your target! Next, let's do a little debugging instead. We'll begin by running openocd again, with some different commands:

In [ ]:
%%bash -s "$PID"

# build firmware
cd ../../hardware/victims/firmware/simpleserial-aes

openocd -f ../../../../openocd/cw_openocd.cfg -c "transport select jtag" -c "ftdi vid_pid 0x2b3e $1" -f "target/stm32f3x.cfg"

**NOTE: The above command will continuously run until you restart this jupyter notebook**

If we want to do some actual debugging, we'll need an interactive terminal. Any terminal with access to the correct gdb and openocd will work, but we'll use Jupyter's terminal here, as it already has everything setup:

![asdf](img/jupyter_terminal.png)

To start debugging, run the following commands in that terminal (you may need to adjust the paths if you don't start in the jupyter folder):

```bash
# try cd hardware/victims/firmware/simpleserial-aes if the command below doesn't work
cd ../hardware/victims/firmware/simpleserial-aes 
arm-none-eabi-gdb simpleserial-aes-CW308_STM32F3.elf -ex "target extended-remote localhost:3333" -ex "monitor reset halt" -ex "load"
```

You should now have an active gdb session up and running:

![](img/gdb_startup.png)

We won't do a full gdb tutorial, but here's some basic commands to get you up and running:
quit
* `help` - prints a useful help screen
* `bt` - prints all stack frames (i.e. all the functions from `main()` down to your current position)
* `s` - step program until it reaches the next source line (moves into function)
* `n` - step program, proceeding through subroutine calls (moves above functions)
* `l` - list source lines around current location

**Warning: the start and run commands may not properly restart the firmware. You can restart from the beginning of your code by running `load` followed by `s`**

![](img/using_gdb.png)